   # Lab 6
**NAME: SRINATH M KRISHNAN | NET ID: SMK220008**

## Dataset description

The dataset for Lab-6 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


## Preparation

Load same data into H2O-3 dataframe.  
Describe imported dataframe.

In [1]:
import h2o
try:
    h2o.cluster().shutdown()
except:
    pass 

In [3]:
# Adjust as per limits on your PC
# Limit to 8 threads and 8GB memory
h2o.init(nthreads=8, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.9" 2023-10-17; OpenJDK Runtime Environment Homebrew (build 17.0.9+0); OpenJDK 64-Bit Server VM Homebrew (build 17.0.9+0, mixed mode, sharing)
  Starting server from /opt/anaconda3/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/x3/5n0yyb7911zg532jx71xlkrr0000gn/T/tmppzo33a4_
  JVM stdout: /var/folders/x3/5n0yyb7911zg532jx71xlkrr0000gn/T/tmppzo33a4_/h2o_srinathmurali_started_from_python.out
  JVM stderr: /var/folders/x3/5n0yyb7911zg532jx71xlkrr0000gn/T/tmppzo33a4_/h2o_srinathmurali_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_srinathmurali_un8eep
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
df_h = h2o.import_file('SBA_loans_lab_6.zip')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
df_h.describe()

Rows:179833
Cols:20

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,Defaulted
type,enum,enum,int,enum,enum,int,int,int,int,int,int,int,int,enum,enum,int,int,int,int,int
mins,,,0.0,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,0.0,0.0,500.0,250.0,0.0
mean,,,53773.700544394,,,399594.7281700244,110.90885988667263,11.384150851067382,1.2807185362326898,9.011527361496501,11.433813593723057,2755.5899473400336,0.759882780134903,,,201926.88862444597,6.553808255436987,193398.5314152574,150058.3506586667,0.17522924046198418
maxs,,,99999.0,,,928120.0,461.0,8500.0,2.0,8800.0,8800.0,99999.0,2.0,,,11446325.0,996262.0,5000000.0,4869000.0,1.0
sigma,,,31182.0827342377,,,263331.7973921778,78.9580696568996,67.30104824719153,0.4520661496065111,246.60707495715346,247.466979825275,12754.79777368782,0.646773081103855,,,290056.8000335532,2367.8805220862832,284469.6765889608,229388.82282767945,0.380164119046022
zeros,,,43,,,40267,161,1369,220,125596,87605,42030,64388,,,44,179829,0,0,148321
missing,3,4,0,316,318,0,0,0,23,0,0,0,0,932,855,0,0,0,0,0
0,Cuyahoga Falls,OH,44224.0,"PNC BANK, NATIONAL ASSOCIATION",OH,332710.0,84.0,3.0,1.0,0.0,3.0,1.0,1.0,N,N,99000.0,0.0,99000.0,49500.0,0.0
1,LAWRENCEVILLE,GA,30043.0,MERRILL LYNCH BANK USA,UT,454390.0,84.0,1.0,2.0,0.0,1.0,1.0,1.0,0,N,100900.0,0.0,108400.0,81300.0,0.0
2,MAPLE GROVE,MN,55369.0,TWIN CITIES-METRO CERT. DEVEL,MN,0.0,240.0,6.0,1.0,4.0,6.0,1.0,0.0,N,N,186000.0,0.0,186000.0,186000.0,0.0


### Set attributes for the categorical  variables - make non-numerical columns "factor" (categorical)

We will be using H2O-3 DRF model. The model can handle categorical variable and missing values.  
We will need to **mark** categorical variables with `asfactor` method

In [11]:
# Choose which columns to encode
cat_columns = ["City","State","Bank","BankState", "NewExist", "RevLineCr","LowDoc","Zip"]
encoded_columns = cat_columns
response = "Defaulted"


df_h[encoded_columns+[response]] = df_h[encoded_columns+[response]].asfactor()










## Split to train/test/validation

In [14]:
train,test,valid = df_h.split_frame(ratios=[.7, .15], seed=123)

## Question 1 

Use H2O-3 to train Distributed Random Forest model with default parameters.  
Documentation: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html#defining-a-drf-model

The solution for the problem is provided.
You need to understand the command and the output in order to proceed to the next question.

In [64]:
from h2o.estimators import H2ORandomForestEstimator

predictors = train.columns
predictors.remove("Defaulted")
print("Predictor columns:", predictors)    
response_col = "Defaulted"

drf = H2ORandomForestEstimator(nfolds=0,seed=123)

drf.train(predictors, response_col, training_frame= train)
model_summary = drf.summary()

Predictor columns: ['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [65]:
model_summary

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,5293376.0,20.0,20.0,20.0,5725.0,7517.0,6730.1


### Model performance metrics on Test dataset

Alternative method to calculate metrics is to use `h2o.make_metrics(pred, actual)` function:   
https://docs.h2o.ai/h2o/latest-stable/h2o-docs/performance-and-prediction.html#computing-model-metrics-from-general-predictions

In [20]:
print("Best accuracy threshold:",drf.model_performance(test).accuracy()[0][0],"\n",
      " Accuracy:",
      drf.model_performance(test).accuracy()[0][1])
print("Best F1 threshold:",drf.model_performance(test).F1()[0][0],"\n",
      " F1:",
      drf.model_performance(test).F1()[0][1])
print("Model AUC:", drf.model_performance(test).auc())
print("Model AUCPR:", drf.model_performance(test).aucpr())
drf.model_performance(test).confusion_matrix()

Best accuracy threshold: 0.4239922069875817 
  Accuracy: 0.9346838459819766
Best F1 threshold: 0.34488910898743647 
  F1: 0.8054782794778516
Model AUC: 0.9562314380121959
Model AUCPR: 0.8733544737137908


,0,1,Error,Rate
0,21272.0,871.0,0.0393,(871.0/22143.0)
1,947.0,3764.0,0.201,(947.0/4711.0)
Total,22219.0,4635.0,0.0677,(1818.0/26854.0)


## Question 2

You are asked to tune the model from question #1. You should review full parameter list for the DRF model documentation: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html#defining-a-drf-model

Pay special attention to the following parameters:
- balance_classes
- ntrees
- max_depth
- early stopping params: stopping_rounds, stopping_metric, stopping_tolerance
- mtries
- sample_rate

Alternative to search for the `ntrees` parameter is to set it to high number, and use early stopping to find optimal number of trees. (We will do it in the next question)

Find best parameters using H2O GridSearch for the following search space.
Modify the example code below as needed to make it work.  
You can experiment with additional options/parameters to see if you can get a better model.
```
# DRF hyperparameters
from h2o.grid.grid_search import H2OGridSearch

# DRF hyperparameters
drf_params2  = {'ntrees': [25,50,100],
               'max_depth': [10,15,20,25]}

# Train and validate a cartesian grid of DRFs
drf_grid2  = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='drf_grid2',
                          hyper_params=drf_params2)

drf_grid2.train(x=predictors,
               y=response_col,
               nfolds=0,
               training_frame=train,
               validation_frame=valid,
               seed=123)
```

You can follow the progress of the Grid search job by using H2O Flow, the URL is shown right after `h2o.init()`.
Answer following questions:
- What is the best set of parameters for the DRF model using AUCPR metric?
- What is the best model performance (AUCPR) on a validation dataset?
- What is the best model performance (AUCPR) on a test dataset?
- Produce Confusion Matrix using best F1 probability threshold

In [54]:
# DRF hyperparameters
from h2o.grid.grid_search import H2OGridSearch

# DRF hyperparameters
drf_params2  = {'ntrees': [25,50,100],
               'max_depth': [10,15,20,25],
                'mtries' : [-2,-1],
                'stopping_metric': ['AUC','AUCPR'],
               }

# Train and validate a cartesian grid of DRFs
drf_grid3  = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='drf_grid3',
                          hyper_params=drf_params2)

drf_grid3.train(x=predictors,
               y=response_col,
               nfolds=0,
               training_frame=train,
               validation_frame=valid,
               seed=123)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,mtries,ntrees,stopping_metric,model_ids,logloss
,15.0,-2.0,100.0,AUC,drf_grid3_model_18,0.1751300
,15.0,-2.0,100.0,AUCPR,drf_grid3_model_42,0.1751300
,10.0,-2.0,100.0,AUC,drf_grid3_model_17,0.1773528
,10.0,-2.0,100.0,AUCPR,drf_grid3_model_41,0.1773528
,10.0,-2.0,50.0,AUCPR,drf_grid3_model_33,0.1777570
,10.0,-2.0,50.0,AUC,drf_grid3_model_9,0.1777570
,10.0,-2.0,25.0,AUC,drf_grid3_model_1,0.1796679
,10.0,-2.0,25.0,AUCPR,drf_grid3_model_25,0.1796679
,15.0,-2.0,50.0,AUC,drf_grid3_model_10,0.1799977
,15.0,-2.0,50.0,AUCPR,drf_grid3_model_34,0.1799977


In [55]:
# Get the grid search results
grid_results = drf_grid3.get_grid(sort_by='aucpr', decreasing=True)

# Best model based on AUCPR
best_model = grid_results.models[0]

# Best parameters
best_params = best_model.actual_params
print("Best parameters for the DRF model using AUCPR metric:", best_params)

# Evaluate model performance on validation and test datasets
aucpr_valid = best_model.model_performance(valid).aucpr()
aucpr_test = best_model.model_performance(test).aucpr()

print("Best model performance (AUCPR) on validation dataset:", aucpr_valid)
print("Best model performance (AUCPR) on test dataset:", aucpr_test)

# Confusion Matrix for the best model at the best F1 threshold
performance = best_model.model_performance(test)
conf_matrix = performance.confusion_matrix(metrics='f1')
print(conf_matrix)


Best parameters for the DRF model using AUCPR metric: {'model_id': 'drf_grid3_model_18', 'training_frame': 'py_3_sid_95b8', 'validation_frame': 'py_5_sid_95b8', 'nfolds': 0, 'keep_cross_validation_models': True, 'keep_cross_validation_predictions': False, 'keep_cross_validation_fold_assignment': False, 'score_each_iteration': False, 'score_tree_interval': 0, 'fold_assignment': None, 'fold_column': None, 'response_column': 'Defaulted', 'ignored_columns': None, 'ignore_const_cols': True, 'offset_column': None, 'weights_column': None, 'balance_classes': False, 'class_sampling_factors': None, 'max_after_balance_size': 5.0, 'max_confusion_matrix_size': 20, 'ntrees': 100, 'max_depth': 15, 'min_rows': 1.0, 'nbins': 20, 'nbins_top_level': 1024, 'nbins_cats': 1024, 'r2_stopping': 1.7976931348623157e+308, 'stopping_rounds': 0, 'stopping_metric': 'AUC', 'stopping_tolerance': 0.001, 'max_runtime_secs': 0.0, 'seed': 123, 'build_tree_one_node': False, 'mtries': -2, 'sample_rate': 0.632, 'sample_rate

In [58]:

predictors = train.columns
predictors.remove("Defaulted")
print("Predictor columns:", predictors)    
response_col = "Defaulted"

drf2 = H2ORandomForestEstimator(nfolds=0,max_depth=15,mtries=-2,ntrees=100,stopping_metric='AUC',seed=123)

drf2.train(predictors, response_col, training_frame= train)
model_summary = drf2.summary()

Predictor columns: ['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [59]:
print("Best accuracy threshold:",drf2.model_performance(test).accuracy()[0][0],"\n",
      " Accuracy:",
      drf2.model_performance(test).accuracy()[0][1])
print("Best F1 threshold:",drf2.model_performance(test).F1()[0][0],"\n",
      " F1:",
      drf2.model_performance(test).F1()[0][1])
print("Model AUC:", drf2.model_performance(test).auc())
print("Model AUCPR(valid):", drf2.model_performance(valid).aucpr())
print("Model AUCPR(test):", drf2.model_performance(test).aucpr())

drf2.model_performance(test).confusion_matrix()


Best accuracy threshold: 0.5431444647053586 
  Accuracy: 0.9385938780070008
Best F1 threshold: 0.4245902270251619 
  F1: 0.8192694547379566
Model AUC: 0.9658818574654644
Model AUCPR(valid): 0.888446843936863
Model AUCPR(test): 0.8913274944232105


,0,1,Error,Rate
0,21278.0,865.0,0.0391,(865.0/22143.0)
1,842.0,3869.0,0.1787,(842.0/4711.0)
Total,22120.0,4734.0,0.0636,(1707.0/26854.0)


## Question 3

You are asked to tune the model from question #1. You should review full parameter list for the DRF model documentation: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html#defining-a-drf-model

Tune model using early stopping criteria by specifying `ntrees`=1000 and setting the parameters required for early stopping. You will need to provide Validation data-frame for early stopping to work correctly.


Alternative to find optimal setting for `ntrees` (number of trees in RF) parameter is to set it to high number, and use early stopping to find optimal number of trees.

Find best parameters using H2O GridSearch for the following search space.
You should experiment with additional options/parameters to see if you can train better models.
The goal is to get a better model than in Q2. Add at least one additional hyper-parameter to tune.
Minimum parameters to tune:
```
drf_params  = {'max_depth': [10,15, 25]}
```
You should review early stopping parameters to be added to the `train` method:
- stopping_rounds
- stopping_metric
- stopping_tolerance

               
Answer following questions:
- What is the best set of parameters for the DRF model using AUCPR metric?
- What is the best model performance (AUCPR) on a validation dataset?
- What is the best model performance (AUCPR) on a test dataset?
- Produce Confusion Matrix using best F1 probability threshold

In [37]:
# Define hyperparameters to search
drf_params = {
    'max_depth': [10, 15, 25],
    'sample_rate': [0.6, 0.8, 1],  # adding sample_rate as an additional hyperparameter to tune
    # Add other parameters here
}

# Define search criteria
search_criteria = {'strategy': "RandomDiscrete", 'max_models': 20, 'seed': 123}

# Initialize H2OGridSearch
drf_grid = H2OGridSearch(
    model=H2ORandomForestEstimator(
        ntrees=1000,  # Set high number of trees and rely on early stopping
        stopping_rounds=5,
        stopping_metric='AUCPR',
        stopping_tolerance=0.0001,
        seed=123,
        score_each_iteration=True
    ),
    hyper_params=drf_params,
    search_criteria=search_criteria,
    grid_id='drf_grid'
)


drf_grid.train(x=predictors,
               y=response_col,
               nfolds=0,
               training_frame=train,
               validation_frame=valid,
               seed=123)

drf Grid Build progress: |███████████████████████████████████████████████████████| 100%e) 100%


,max_depth,ntrees,stopping_metric,stopping_tolerance,model_ids,logloss
,20.0,1000.0,AUCPR,0.001,drf_grid_model_3,0.1817736
,20.0,1000.0,AUCPR,0.0001,drf_grid_model_6,0.1817736
,15.0,1000.0,AUCPR,0.001,drf_grid_model_2,0.1935741
,15.0,1000.0,AUCPR,0.0001,drf_grid_model_5,0.1935741
,25.0,49.0,AUCPR,0.0001,drf_grid_model_10,0.1944238
,15.0,49.0,AUCPR,0.0001,drf_grid_model_15,0.1949279
,15.0,49.0,AUCPR,0.0001,drf_grid_model_12,0.1979517
,25.0,48.0,AUCPR,0.0001,drf_grid_model_7,0.1987382
,15.0,28.0,AUCPR,0.0001,drf_grid_model_14,0.1990763
,25.0,50.0,AUCPR,0.0001,drf_grid_model_11,0.2010267


In [39]:
# Get the grid search results
grid_results = drf_grid.get_grid(sort_by='aucpr', decreasing=True)

# Best model based on AUCPR
best_model = grid_results.models[0]

# Best parameters
best_params = best_model.actual_params
print("Best parameters for the DRF model using AUCPR metric:", best_params)

# Evaluate model performance on validation and test datasets
aucpr_valid = best_model.model_performance(valid).aucpr()
aucpr_test = best_model.model_performance(test).aucpr()

print("Best model performance (AUCPR) on validation dataset:", aucpr_valid)
print("Best model performance (AUCPR) on test dataset:", aucpr_test)

# Confusion Matrix for the best model at the best F1 threshold
performance = best_model.model_performance(test)
conf_matrix = performance.confusion_matrix(metrics='f1')
print(conf_matrix)


Best parameters for the DRF model using AUCPR metric: {'model_id': 'drf_grid_model_3', 'training_frame': 'py_3_sid_95b8', 'validation_frame': 'py_5_sid_95b8', 'nfolds': 0, 'keep_cross_validation_models': True, 'keep_cross_validation_predictions': False, 'keep_cross_validation_fold_assignment': False, 'score_each_iteration': False, 'score_tree_interval': 0, 'fold_assignment': None, 'fold_column': None, 'response_column': 'Defaulted', 'ignored_columns': None, 'ignore_const_cols': True, 'offset_column': None, 'weights_column': None, 'balance_classes': False, 'class_sampling_factors': None, 'max_after_balance_size': 5.0, 'max_confusion_matrix_size': 20, 'ntrees': 1000, 'max_depth': 20, 'min_rows': 1.0, 'nbins': 20, 'nbins_top_level': 1024, 'nbins_cats': 1024, 'r2_stopping': 1.7976931348623157e+308, 'stopping_rounds': 0, 'stopping_metric': 'AUCPR', 'stopping_tolerance': 0.001, 'max_runtime_secs': 0.0, 'seed': 123, 'build_tree_one_node': False, 'mtries': -1, 'sample_rate': 0.632, 'sample_rat

## Question 4

Use best model parameters from the GridSearch in question #3. Train new model as following:
- Train model on a combined dataset: train+validation
- Don't use cross-validation - `nfolds=0`
- Evaluate trained model on test dataset
- Compare AUCPR (test dataset) metrics from the question #2 and #3

In [48]:
predictors = train.columns
predictors.remove("Defaulted")
print("Predictor columns:", predictors)    
response_col = "Defaulted"

drf4 = H2ORandomForestEstimator(nfolds=0,max_depth=20,ntrees=1000,stopping_metric='AUCPR',stopping_tolerance=0.001,seed=123)

drf4.train(predictors, response_col, training_frame= train)
model_summary = drf4.summary()

Predictor columns: ['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [50]:
print("Best accuracy threshold:",drf4.model_performance(test).accuracy()[0][0],"\n",
      " Accuracy:",
      drf4.model_performance(test).accuracy()[0][1])
print("Best F1 threshold:",drf4.model_performance(test).F1()[0][0],"\n",
      " F1:",
      drf2.model_performance(test).F1()[0][1])
print("Model AUC:", drf4.model_performance(test).auc())
print("Model AUCPR(valid):", drf4.model_performance(valid).aucpr())
print("Model AUCPR(test):", drf4.model_performance(test).aucpr())

drf4.model_performance(test).confusion_matrix()

Best accuracy threshold: 0.419178338345344 
  Accuracy: 0.9366947195948462
Best F1 threshold: 0.3548650235280188 
  F1: 0.8192694547379566
Model AUC: 0.9616693984229964
Model AUCPR(valid): 0.8835800980769156
Model AUCPR(test): 0.8827062088221262


,0,1,Error,Rate
0,21351.0,792.0,0.0358,(792.0/22143.0)
1,952.0,3759.0,0.2021,(952.0/4711.0)
Total,22303.0,4551.0,0.0649,(1744.0/26854.0)


In [62]:
print("Model AUCPR(test) Question 2:", drf2.model_performance(test).aucpr())
print("Model AUCPR(test) Question 3:", drf4.model_performance(test).aucpr())

Model AUCPR(test) Question 2: 0.8913274944232105
Model AUCPR(test) Question 3: 0.8827062088221262


## Question 5

Review following documentation section first: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/cross-validation.html

- Train DRF model with the parameters found in the Question 3.
- Use full data to train the model.
- Use cross-validation to produce reliable metrics for the model performance
- Produce model metrics using cross-validation 

**Note**: understand why the model training takes significantly longer than before we used cross-validation

In [70]:
# Best parameters found from previous question
best_params = {
    'max_depth': 20,  # Example parameter, replace with your actual best parameters
    'ntrees': 1000,  # Set this to a high value, assuming early stopping will find the optimal number
    # Include other parameters as necessary
}

# Initialize and train the DRF model with cross-validation
drf_model = H2ORandomForestEstimator(
    **best_params,
    nfolds=2,  # Use 5-fold cross-validation
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=1234  # Ensure reproducibility
)

drf_model.train(x=predictors, y=response, training_frame=df_h)


drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1708662317593_3263


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    1000               1000                        1.388e+08              20           20           20            6924          10320         8756.75

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.04955645938042829
RMSE: 0.22261280147473167
LogLoss: 0.17828060425824427
Mean Per-Class Error: 0.11425024509213282
AUC: 0.9629723794443907
AUCPR: 0.8838614495794228
Gini: 0.9259447588887815

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3556466325508069
       0       1      Error    Rate
-----  ------  -----  -------  ------------------
0      143127  5194   0.035    (5194.0/148321.0)
1      6097    25415  0.1935   (6097.0/31512.0)
Total  149224  30609  0.0628   (11291.0/179833.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.355647     0.818242  193
max f2                       0.210674     0.836735  244
max f0point5                 0.55743      0.858746  136
max accuracy                 0.418055     0.93896   174
max precision                0.984825     0.994609  2
max recall                   0.00253775   1         397
max specificity              0.991526     0.999987  0
max absolute_mcc             0.418055     0.781617  174
max min_per_class_accuracy   0.167764     0.89696   264
max mean_per_class_accuracy  0.210674     0.899916  244
max tns                      0.991526     148319    0
max fns                      0.991526     31441     0
max fps                      0.00107944   148321    399
max tps                      0.00253775   31512     397
max tnr                      0.991526     0.999987  0
max fnr                      0.991526     0.997747  0
max fpr                      0.00107944   1         399
max tpr                      0.00253775   1         397

Gains/Lift Table: Avg response rate: 17.52 %, avg score: 17.43 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100037                   0.968215           5.61482     5.61482            0.98388          0.977771    0.98388                     0.977771            0.0561691       0.0561691                  461.482   461.482            0.0559736
2        0.0200019                   0.952502           5.54494     5.57989            0.971635         0.960419    0.977759                    0.969098            0.0554392       0.111608                   454.494   457.989            0.111069
3        0.0300001                   0.936847           5.53224     5.56401            0.96941          0.944702    0.974977                    0.960967            0.0553123       0.166921                   453.224   456.401            0.16601
4        0.0400038                   0.920587           5.48158     5.5434             0.960534         0.928717    0.971365                    0.952902            0.0548363       0.221757                   448.158   454.34             0.220368
5        0.0500019                   0.901677           5.47828     5.53038            0.959956     

In [72]:
# Print the cross-validated metrics
cv_model_performance = drf_model.cross_validation_metrics_summary().as_data_frame()
print(cv_model_performance)


                                    mean         sd   cv_1_valid   cv_2_valid
0                  accuracy     0.933911   0.000196     0.933772     0.934050
1                       auc     0.958275   0.000446     0.957960     0.958591
2                       err     0.066089   0.000196     0.066228     0.065950
3                 err_count  5942.500000  17.677670  5955.000000  5930.000000
4                  f0point5     0.818021   0.001665     0.819199     0.816844
5                        f1     0.807302   0.001956     0.805919     0.808685
6                        f2     0.796875   0.005392     0.793062     0.800687
7            lift_top_group     5.605356   0.035914     5.630751     5.579961
8                   logloss     0.187941   0.000656     0.188405     0.187477
9       max_per_class_error     0.209920   0.007584     0.215283     0.204557
10                      mcc     0.767723   0.001822     0.766434     0.769011
11  mean_per_class_accuracy     0.877274   0.003106     0.875078

# Why cross validation takes more time to run?

Using cross-validation significantly increases the model training time because the training process is repeated multiple times (once for each fold). For example, with 5-fold cross-validation, the model is trained five separate times on different subsets of the data. Each training iteration uses (n-1)/n of the full dataset as the training set and 1/n as the validation set, where n is the number of folds. This process provides a more robust estimation of the model's performance on unseen data by averaging the metrics across all folds but at the cost of increased computational time.